In [1]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.0 MB/s eta 0:00:00


In [6]:
# %% [markdown]
# # 🏥 HealthGuardian AI - Versão Colab
# Sistema Inteligente de Triagem Médica

# %%
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import os
from IPython.display import display, Markdown
import ipywidgets as widgets

# Carregar ou criar dados de treinamento
def load_data():
    sample_data = {
        'symptoms': [
            'febre dor_cabeça coriza', 'febre_alta calafrios dor_corpo',
            'dor_peito falta_ar', 'dor_abdominal nausea vômito',
            'tosse_seca falta_ar', 'dor_cabeça_leve', 'manchas_pele febre',
            'dor_garganta tosse', 'tontura visão_turva', 'dor_ouvido febre'
        ],
        'condition': [
            'gripe', 'influenza', 'emergência_cardíaca', 'gastroenterite',
            'covid_suspeita', 'cefaléia', 'dengue_suspeita', 'amigdalite',
            'pressao_arterial', 'otite'
        ],
        'urgency': ['low', 'medium', 'high', 'medium', 'high',
                   'low', 'high', 'medium', 'high', 'medium'],
        'recommendations': [
            'repouso hidratação monitorar_temperatura', 'repouso hidratação medicação_febre consulta_24h',
            'procurar_atendimento_imediato', 'dieta_leve hidratação consulta_48h',
            'isolamento teste_covid consulta_urgente', 'repouso analgésico',
            'repouso hidratacao procurar_urgente', 'gargarejo analgésico consulta_médica',
            'medir_pressão procurar_urgente', 'analgésico consulta_médica'
        ]
    }
    return pd.DataFrame(sample_data)

# Treinar modelo
def train_model(df):
    vectorizer = TfidfVectorizer(max_features=100)
    X = vectorizer.fit_transform(df['symptoms'])
    y_condition = df['condition']
    y_urgency = df['urgency']

    model_condition = RandomForestClassifier(n_estimators=50, random_state=42)
    model_urgency = RandomForestClassifier(n_estimators=50, random_state=42)

    model_condition.fit(X, y_condition)
    model_urgency.fit(X, y_urgency)

    return vectorizer, model_condition, model_urgency

# Carregar dados e treinar modelo
df = load_data()
vectorizer, model_condition, model_urgency = train_model(df)

# %%
# Interface interativa
display(Markdown("## 📋 Descreva Seus Sintomas"))

symptoms_input = widgets.Textarea(
    value='',
    placeholder='Exemplo: febre, dor de cabeça, coriza, náusea...',
    description='Sintomas:',
    layout=widgets.Layout(width='80%', height='100px')
)

analyze_button = widgets.Button(description="🔍 Analisar Sintomas", button_style='primary')

output = widgets.Output()

def on_analyze_click(b):
    with output:
        output.clear_output()
        symptoms_text = symptoms_input.value.strip()

        if symptoms_text:
            # Pré-processar e prever
            symptoms_vec = vectorizer.transform([symptoms_text])

            condition_pred = model_condition.predict(symptoms_vec)[0]
            urgency_pred = model_urgency.predict(symptoms_vec)[0]

            # Exibir resultados
            display(Markdown("---"))
            display(Markdown("## 📊 Resultados da Triagem"))

            # Nível de urgência
            urgency_colors = {'low': '🟢', 'medium': '🟡', 'high': '🔴'}
            urgency_descs = {
                'low': 'BAIXA - Consulta em 48-72h',
                'medium': 'MÉDIA - Consulta em 24-48h',
                'high': 'ALTA - Procurar atendimento imediato'
            }

            display(Markdown(f"### {urgency_colors[urgency_pred]} NÍVEL DE URGÊNCIA: {urgency_descs[urgency_pred]}"))
            display(Markdown(f"### 🩺 POSSÍVEIS CONDIÇÕES: {condition_pred.replace('_', ' ').title()}"))

            # Aviso
            display(Markdown("---"))
            display(Markdown("""
            ⚠️ **AVISO IMPORTANTE:**
            Este é um sistema de triagem preliminar e **NÃO SUBSTITUI**
            a avaliação de um profissional de saúde.
            Sempre consulte um médico para diagnóstico preciso.
            """))
        else:
            display(Markdown("❌ **Por favor, descreva seus sintomas para análise.**"))

analyze_button.on_click(on_analyze_click)

display(symptoms_input)
display(analyze_button)
display(output)

## 📋 Descreva Seus Sintomas

Textarea(value='', description='Sintomas:', layout=Layout(height='100px', width='80%'), placeholder='Exemplo: …

Button(button_style='primary', description='🔍 Analisar Sintomas', style=ButtonStyle())

Output()